<a href="https://colab.research.google.com/github/mohameddhameem/algorithmic-trading/blob/main/Intro_to_Q_Table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

** Some basic terminology

State: s

Available actions: a

Greedy policy: $\epsilon$

Learning rate: $\alpha$ 

Discount factor: $\gamma$

Maximum episodes

### Sudo algo

Initialize Q(s,a) arbirarily

For each episode, repeat:



1.   Initialize state s
2.   Choose action a from state s using policy derived from Q value
3.   Repeat (for each step of episode):

     a.   Take action a and then observe r, s’(next state)

     b.   Choose action a’ from state s’ using policy derived from Q value
     c.   update Q value by $Q(s, a) \leftarrow Q(s, a) + \alpha \cdot (r + \gamma Q(s’,a’) - Q(s,a))$

     d.    update s by s’, a by a’


stop till s reaches termination

In [3]:
!pip install gym[toy_text]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.3 MB 94 kB/s 


In [5]:
!pip install cmake 'gym[atari]' scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.5 MB/s 


In [6]:
import gym
import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

env = gym.make("Taxi-v3").env

In [7]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


In [8]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328


In [10]:
# reward table
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

### With No - reinforment learning

In [11]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

/usr/local/lib/python3.7/dist-packages/gym/core.py:44: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  "The argument mode in render method is deprecated; "


Timesteps taken: 4377
Penalties incurred: 1363


In [13]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1350
State: 288
Action: 5
Reward: -10


KeyboardInterrupt: ignored

above indicates we are not learning from the past.

## Basic Q-Learning with Q-Table

In [14]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [15]:
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [16]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 1min 36s, sys: 14.3 s, total: 1min 50s
Wall time: 1min 39s


In [17]:
q_table

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ -2.41837066,  -2.3639511 ,  -2.41837057,  -2.3639511 ,
         -2.27325184, -11.36394981],
       [ -1.87014399,  -1.45024004,  -1.87014389,  -1.45024   ,
         -0.7504    , -10.45023871],
       ...,
       [ -1.01068103,   0.41599979,  -1.07373389,  -1.20371567,
         -4.64011308,  -4.0928417 ],
       [ -2.14700272,  -2.12205445,  -2.14529409,  -2.12205266,
         -5.11864301,  -3.68741415],
       [  3.96132488,   1.25461444,   0.90349009,  11.        ,
         -2.25250427,  -2.72236933]])